In [ ]:
import tensorflow as tf

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding,Bidirectional,LSTM,Dense,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [ ]:
df=pd.read_csv("/content/evaluation.csv",sep=';',engine='python',on_bad_lines='skip')

In [ ]:
df.isnull().sum()

,0
Unnamed: 0,0
title,0
text,0
label,1


In [ ]:
df=df.dropna()

In [ ]:
messages=df.copy()
y=messages['label']

In [ ]:
df.head()

,Unnamed: 0,title,text,label
0,0,"Sanders back in U.S. Senate, blasts 'coloniali...",WASHINGTON (Reuters) - Democratic U.S. preside...,1.0
1,1,Kremlin: Syria peoples' congress being 'active...,MOSCOW (Reuters) - A proposal to convene a con...,1.0
2,2,Oregon Cop Convicted Of Shattering Biker’s Co...,"In a baffling fit of rage, an Oregon State Pol...",0.0
3,3,Twitter Erupts With Glee Over #CruzSexScandal...,The last thing any politician running for the ...,0.0
4,4,MUST WATCH VIDEO: Obama Tries To Trash Trump B...,This is too good to miss! Mr. Teleprompter did...,0.0


In [ ]:
messages.reset_index(inplace=True)

In [ ]:
df['label'].value_counts()

,count
label,
1.0,1305
0.0,1189


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words=stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from nltk.stem import PorterStemmer
ps=PorterStemmer()
import re
corpus=[]
for i in range(0,len(messages)):
  review=re.sub('[^a-zA-Z]',' ',messages['title'][i])
  review=review.lower().split()

  review=[ps.stem(word) for word in review if word not in stop_words]
  review=' '.join(review)
  corpus.append(review)

In [ ]:
voc_size=5000
sent_length=30

In [ ]:
# one_hot_rep=[one_hot(word,voc_size) for word in corpus]
# one_hot_rep
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer(voc_size)
tokenizer.fit_on_texts(corpus)
sequences=tokenizer.texts_to_sequences(corpus)

In [ ]:
embedded_docs=pad_sequences(sequences,padding='pre',maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 2478,  454,  455],
       [   0,    0,    0, ...,  125,  744,  197],
       [   0,    0,    0, ..., 1743, 2480,    2],
       ...,
       [   0,    0,    0, ...,  243,  579,    3],
       [   0,    0,    0, ..., 2453,   14,    8],
       [   0,    0,    0, ...,   14,    8,  246]], dtype=int32)

In [ ]:
# dim=300
# model=Sequential([])
# model.add(Embedding(voc_size,dim))
# model.build((None,sent_length))
# model.add(Dropout(0.3))
# model.add(Bidirectional(LSTM(155)))
# model.add(Dropout(0.3))
# model.add(Dense(128,activation='relu'))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# model.summary()
dim=300
model=Sequential([
  Embedding(input_dim=voc_size,output_dim=dim),
  Dropout(0.3),
  Bidirectional(LSTM(155)),
  Dropout(0.3),
  Dense(128,activation='relu'),
  Dense(1,activation='sigmoid')
])
model.build(input_shape=(None,sent_length))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 30, 300)        │     1,500,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 30, 300)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 310)            │       565,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 310)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        39,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,105,377 (8.03 MB)

 Trainable params: 2,105,377 (8.03 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import numpy as np
y_final=np.array(y)
x_final=np.array(embedded_docs)

In [ ]:
print(y[:5])  # show some labels
print(type(y))  # should be pandas.Series or list


0    1.0
1    1.0
2    0.0
3    0.0
4    0.0
Name: label, dtype: float64
<class 'pandas.core.series.Series'>


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.33,random_state=98)

In [ ]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=25)

Epoch 1/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - accuracy: 0.6656 - loss: 0.6107 - val_accuracy: 0.8774 - val_loss: 0.3188
Epoch 2/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9487 - loss: 0.1290 - val_accuracy: 0.8786 - val_loss: 0.3030
Epoch 3/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9905 - loss: 0.0451 - val_accuracy: 0.8920 - val_loss: 0.3630
Epoch 4/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9955 - loss: 0.0137 - val_accuracy: 0.8883 - val_loss: 0.3883
Epoch 5/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9976 - loss: 0.0089 - val_accuracy: 0.8786 - val_loss: 0.4723
Epoch 6/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9986 - loss: 0.0048 - val_accuracy: 0.8786 - val_loss: 0.4742
Epoch 7/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9999 - loss: 9.5401e-04 - val_accuracy: 0.8750 - val_loss: 0.5147
Epoch 8/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9998 - loss: 0.0016 - val_accuracy: 0.8726

In [ ]:
ans=model.predict(x_test)

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


ans=model.predict(x_test)
ans=np.where(ans>0.5,1,0)
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
accuracy_score(y_test,ans)
confusion_matrix(y_test,ans)
print(classification_report(y_test,ans))

In [ ]:
ans=np.where(ans>0.5,1,0)

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
accuracy_score(y_test,ans)

0.8859223300970874

In [ ]:
confusion_matrix(y_test,ans)

array([[339,  43],
       [ 51, 391]])

In [ ]:
print(classification_report(y_test,ans))

              precision    recall  f1-score   support

         0.0       0.87      0.89      0.88       382
         1.0       0.90      0.88      0.89       442

    accuracy                           0.89       824
   macro avg       0.89      0.89      0.89       824
weighted avg       0.89      0.89      0.89       824

